# ETL del dataset user_reviews

Importacion de librerias 

In [24]:
import pandas as pd
import gzip

Carga de dataset

In [38]:
with gzip.open('../Jsons/user_reviews.json.gz', 'rt', encoding='utf-8') as f:
    df = [eval(line) for line in f]

user_reviews = pd.DataFrame(df)

Visualizacion de los primeros 5 registros

In [39]:
user_reviews.head() 

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Eliminacion de columas iniecesarias

In [40]:
user_reviews = user_reviews.drop(columns=['user_url']) 

Cuento los nulos de cada columa para ver como tratar los datos

In [28]:
user_reviews.isnull().sum() 

user_id    0
reviews    0
dtype: int64

Desanidado

In [55]:
df = user_reviews.explode('reviews') # creo un archivo con la columan que me interesa ingresar
dfreviewsOpen = pd.concat([df.drop(['reviews'], axis=1), df['reviews'].apply(pd.Series)], axis=1) #con esta expresion  creo un nuevo df donde esta la info desanidada en columnas

Eliminacion de columas iniecesarias

In [56]:
df = dfreviewsOpen.drop(['funny','last_edited','helpful',0], axis=1) #Eliminacion de columas iniecesarias

Eliminar la palabra posted de la columna posted

In [57]:
df['posted'] = df['posted'].str.replace('Posted', '')

Tratamiento para formatear las columnas 'posted'

In [59]:
# Reemplazar los valores no válidos por NaN
df["posted"] = pd.to_datetime(df["posted"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos
df_filtered = df.dropna(subset=["posted"])
# Convertir la columna "release_date" a tipo datetime
df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])
# Rellenando los valores faltantes en las columna'release_date' con la moda
columns_fill_mode = ['posted']
for col in columns_fill_mode:
    df[col].fillna(df[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date'
df['posted_year'] = df['posted'].dt.year.astype('Int64')
df.drop('posted', axis=1, inplace=True)

C:\Users\gerar\AppData\Local\Temp\ipykernel_10668\4269440846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])


In [61]:
df.to_csv('../Data/df_reviews.csv', index=False)